In [1]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
xgb.__version__

'1.5.1'

In [2]:
!nvidia-smi

Thu Dec  2 12:57:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   38C    P8    28W / 280W |  23171MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [3]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=[0,1])
client = Client(cluster)

distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [4]:
!nvidia-smi

Thu Dec  2 12:57:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   39C    P2    65W / 280W |  23489MiB / 24220MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [6]:
%%time
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold = dd.read_csv(f'../input/xgtrain_fold_{i}.csv')
    val_fold = dd.read_csv(f'../input/xgval_fold_{i}.csv')
    
    train_fold = train_fold.replace([np.inf, -np.inf], np.nan)
    val_fold = val_fold.replace([np.inf, -np.inf], np.nan)
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4
CPU times: user 4.25 s, sys: 78.9 ms, total: 4.33 s
Wall time: 4.2 s


In [7]:
application_train = pd.read_csv('../input/application_train.csv')
target = application_train.TARGET.values
train_oof = np.zeros((application_train.shape[0],))

In [8]:
num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }
    
    #start_time = time()
    kf = StratifiedKFold(5, shuffle=True, random_state=1974)

    for i, (train_index, test_index) in enumerate(kf.split(application_train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i], train_ys[i])
        dval = xgb.dask.DaskDMatrix(client, val_folds[i], val_ys[i])
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions


    roc = roc_auc_score(target, train_oof)
    
    return roc

In [9]:
study = optuna.create_study(direction='maximize')

[I 2021-12-02 13:00:32,850] A new study created in memory with name: no-name-d1fa903c-ed0e-4e86-bc32-3ed457a4674d


In [10]:
%%time
study.optimize(objective, n_trials=1)

[13:01:35] task [xgboost.dask]:tcp://192.168.1.200:44387 got new rank 0
[13:01:35] task [xgboost.dask]:tcp://192.168.1.200:44983 got new rank 1
distributed.worker - WARNING - Compute Failed
Function:  dispatched_train
args:      ('tcp://192.168.1.200:44387', [b'DMLC_NUM_WORKER=2', b'DMLC_TRACKER_URI=192.168.1.200', b'DMLC_TRACKER_PORT=9091', b'DMLC_TASK_ID=[xgboost.dask]:tcp://192.168.1.200:44387'], {'feature_names': None, 'feature_types': None, 'feature_weights': None, 'meta_names': ['labels'], 'missing': None, 'enable_categorical': False, 'parts': [(      ACT_AMT_ANNUITY_MAX  ...  total_credit_None
0                     0.0  ...         865055.565
1                     NaN  ...                NaN
2                     NaN  ...         146250.000
3                     NaN  ...         468445.500
4                     NaN  ...        4800811.500
...                   ...  ...                ...
6562                  NaN  ...         508927.500
6563                  NaN  ...        1098

XGBoostError: [13:01:44] ../src/tree/updater_gpu_hist.cu:770: Exception in gpu_hist: [13:01:44] ../src/c_api/../data/../common/device_helpers.cuh:428: Memory allocation error on worker 0: Caching allocator
- Free memory: 405274624
- Requested memory: 271294719

Stack trace:
  [bt] (0) /home/tunguz/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x3147d9) [0x7ff7221c77d9]
  [bt] (1) /home/tunguz/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x3184fb) [0x7ff7221cb4fb]
  [bt] (2) /home/tunguz/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x3262a1) [0x7ff7221d92a1]
  [bt] (3) /home/tunguz/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x327e8d) [0x7ff7221dae8d]
  [bt] (4) /home/tunguz/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x37fd40) [0x7ff722232d40]
  [bt] (5) /home/tunguz/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x38036e) [0x7ff72223336e]
  [bt] (6) /home/tunguz/.local/lib/python3.8/si

distributed.worker - WARNING - Compute Failed
Function:  dispatched_train
args:      ('tcp://192.168.1.200:44983', [b'DMLC_NUM_WORKER=2', b'DMLC_TRACKER_URI=192.168.1.200', b'DMLC_TRACKER_PORT=9091', b'DMLC_TASK_ID=[xgboost.dask]:tcp://192.168.1.200:44983'], {'feature_names': None, 'feature_types': None, 'feature_weights': None, 'meta_names': ['labels'], 'missing': None, 'enable_categorical': False, 'parts': [(      ACT_AMT_ANNUITY_MAX  ...  total_credit_None
0                     NaN  ...        2413192.500
1                     NaN  ...        2092986.000
2                     NaN  ...         131395.500
3                     NaN  ...        1802266.830
4                     NaN  ...          79156.125
...                   ...  ...                ...
6570                  NaN  ...         488983.725
6571                  NaN  ...         213781.500
6572                  NaN  ...        6467192.505
6573                  NaN  ...         585000.000
6574              36504.0  ...      